# CHi Squared Test for Independece
- Well feed out chi2 function 2 series
- output from chi2 function that we care about is the p value
## process
1. set your alpha. state our null hypothesis. the chi2 null hypothesis is:
- theres no relationship between A and B
- A and B categories are independent
2. run an 'obeserved = pd.crosstab(df.A,df.B)
3. compare your p to a. if p < a then we reject the null, we have the evidence supporting the alternate hypothesis.

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import env
import pandas as pd
from scipy import stats
from pydataset import data

mpg = data('mpg')
mpg['transmission'] = mpg.trans.str[:-4] # a little cleaning goes a long way
mpg.head()


,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,transmission
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,auto
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,manual
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,manual
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,auto
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,auto


In [2]:
n = mpg.shape[0]

transmission_proportions = mpg.transmission.value_counts() / n
transmission_proportions


auto      0.67094
manual    0.32906
Name: transmission, dtype: float64

In [3]:
drive_proportions = mpg.drv.value_counts() / n
drive_proportions

f    0.452991
4    0.440171
r    0.106838
Name: drv, dtype: float64

In [4]:
expected = pd.DataFrame()

for transmission_group, t_prop in transmission_proportions.iteritems():
    for drive_group, d_prop in drive_proportions.iteritems():
        expected.loc[drive_group, transmission_group] = t_prop * d_prop

expected.sort_index(inplace=True)
expected


,auto,manual
4,0.295328,0.144843
f,0.303930,0.149061
r,0.071682,0.035156


In [5]:
expected *= n
expected


,auto,manual
4,69.106838,33.893162
f,71.119658,34.880342
r,16.773504,8.226496


In [6]:
observed = pd.crosstab(mpg.drv, mpg.transmission)
observed


transmission,auto,manual
drv,,
4,75,28
f,65,41
r,17,8


In [7]:
chi2 = ((observed - expected)**2 / expected).values.sum()
chi2


3.136769245971112

In [9]:
nrows, ncols = observed.shape

degrees_of_freedom = (nrows - 1) * (ncols - 1)

In [10]:
stats.chi2(degrees_of_freedom).sf(chi2)


0.20838152534979645

# !

In [13]:
# hard way
index = ['Churn', 'No Churn']
columns = ['Product A', 'Product B']

observed = pd.DataFrame([[100, 50], [120, 28]], index=index, columns=columns)
n = observed.values.sum()

expected = pd.DataFrame([[.372, .132], [.367, .130]], index=index, columns=columns) * n

chi2 = ((observed - expected)**2 / expected).values.sum()

nrows, ncols = observed.shape

degrees_of_freedom = (nrows - 1) * (ncols - 1)

p = stats.chi2(degrees_of_freedom).sf(chi2)

print('Observed')
print(observed)
print('---\nExpected')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')


Observed
          Product A  Product B
Churn           100         50
No Churn        120         28
---
Expected
          Product A  Product B
Churn       110.856     39.336
No Churn    109.366     38.740
---

chi^2 = 7.9656
p     = 0.0048


In [15]:
#observed = pd.crosstab(mpg.drv, mpg.transmission)
#observed

# easy way
chi2, p, degf, expected = stats.chi2_contingency(observed) #code !!!!!!!

print('Observed\n')
print(observed.values)
print('---\nExpected\n')
print(expected)
print('---\n')
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')


Observed

[[100  50]
 [120  28]]
---
Expected

[[110.73825503  39.26174497]
 [109.26174497  38.73825503]]
---

chi^2 = 7.2817
p     = 0.0070


# !

In [18]:
#example
df = data("tips")

In [19]:
observed = pd.crosstab(df.smoker, df.time)
observed

time,Dinner,Lunch
smoker,,
No,106,45
Yes,70,23


In [20]:
alpha = .05

In [21]:
chi2, p, degf, expected = stats.chi2_contingency(observed)
p

0.4771485672079724

In [22]:
observed = pd.crosstab(df.sex, df.day)
observed

day,Fri,Sat,Sun,Thur
sex,,,,
Female,9,28,18,32
Male,10,59,58,30


In [23]:
chi2, p, degf, expected = stats.chi2_contingency(observed)
p

0.004180302092822257

In [26]:
#time and day
# null hypothesis : time and day are independent

observed = pd.crosstab(df.time, df.day)
observed

day,Fri,Sat,Sun,Thur
time,,,,
Dinner,12,87,76,1
Lunch,7,0,0,61


In [27]:
# there are 0 
chi2, p, degf, expected = stats.chi2_contingency(observed)
p

8.449897551777147e-47

In [29]:
if p < alpha:
    print("reject the null")
else:
    print("fail to reject the null")

reject the null


In [33]:
df.day

1       Sun
2       Sun
3       Sun
4       Sun
5       Sun
       ... 
240     Sat
241     Sat
242     Sat
243     Sat
244    Thur
Name: day, Length: 244, dtype: object